In [ ]:
from google.cloud import storage

bucket_name = 'nyt-news-bucket' # Replace with your actual GCS bucket name
source_file_name = '/content/텍마/news_posclean.csv'
destination_blob_name = 'news_posclean.csv' # The name of the file in the GCS bucket

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

print(f"File {source_file_name} uploaded to {bucket_name}/{destination_blob_name}.")

File /content/텍마/news_posclean.csv uploaded to nyt-news-bucket/news_posclean.csv.


## 뉴욕타임즈 나누기


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import json
import spacy
import pandas as pd
from tqdm.auto import tqdm

nlp = spacy.load("en_core_web_sm")
# Use Python engine and warn on bad lines for more robust parsing of potentially malformed CSV
nyt_raw = pd.read_csv("/content/텍마/nyt_raw_all.csv", engine='python', on_bad_lines='warn')
len(nyt_raw)

/tmp/ipython-input-976651144.py:8: ParserWarning: Skipping line 11737: field larger than field limit (131072)

  nyt_raw = pd.read_csv("/content/텍마/nyt_raw_all.csv", engine='python', on_bad_lines='warn')


34462

In [ ]:
nyt_raw.head()

,doc_id,doc_type,date,title,source,language,raw_text,web_url,section_name,news_desk
0,NEWS_1993-01-31_6885,news,1993-01-31,Publishing At Their Peril,The New York Times,en,Publishing At Their Peril Prime Minister John ...,https://www.nytimes.com/1993/01/31/weekinrevie...,Week in Review,Week in Review Desk
1,NEWS_1993-01-31_6886,news,1993-01-31,County's Data Systems At Yonkers's Fingertips,The New York Times,en,County's Data Systems At Yonkers's Fingertips ...,https://www.nytimes.com/1993/01/31/nyregion/co...,New York,Westchester Weekly Desk
2,NEWS_1993-01-31_6901,news,1993-01-31,Hunt for Highway Killer Is Undiminished,The New York Times,en,Hunt for Highway Killer Is Undiminished Along ...,https://www.nytimes.com/1993/01/31/nyregion/hu...,New York,Metropolitan Desk
3,NEWS_1993-01-31_6917,news,1993-01-31,Viewpoints;\nThe Monetary Path to Full Recovery,The New York Times,en,Viewpoints;\nThe Monetary Path to Full Recover...,https://www.nytimes.com/1993/01/31/business/vi...,Business Day,Financial Desk
4,NEWS_1993-01-31_6922,news,1993-01-31,"Jeremy Weithas, M. J. Minnetian",The New York Times,en,"Jeremy Weithas, M. J. Minnetian Jeremy Anne We...",https://www.nytimes.com/1993/01/31/style/weddi...,Style,Society Desk


In [ ]:
import pandas as pd

raw = pd.read_csv("/content/텍마/nyt_raw_all.csv", engine='python', on_bad_lines='warn')

# 1) 뉴스 카테고리중 경제/비즈니스 관련 섹션만 남기기
keep_sections = [
    "Business Day", "Business", "Economy", "Your Money",
    "Markets", "DealBook", "U.S. Economy"
]
raw = raw[raw["section_name"].isin(keep_sections)]

# 2) 정책어가 하나라도 들어가는 문서만 남기기
POLICY_TERMS_STRONG = [
    "interest rate", "interest rates", "policy rate", "base rate",
    "benchmark rate", "key rate", "monetary policy",
    "federal reserve", "fomc", "fed funds rate",
    "rate hike", "rate cut", "tightening", "easing",
    "forward guidance", "policy guidance", "rate guidance",
    "dot plot", "summary of economic projections", "sep"
]

def contains_any(text, terms):
    if not isinstance(text, str):
        return False
    low = text.lower()
    return any(t in low for t in terms)

raw = raw[raw["raw_text"].apply(lambda x: contains_any(x, POLICY_TERMS_STRONG))]

# 3) 이렇게 1차 필터링된 문서만 따로 저장
raw.to_csv("nyt_raw_core_candidates.csv", index=False)
len(raw)

/tmp/ipython-input-2331950264.py:3: ParserWarning: Skipping line 11737: field larger than field limit (131072)

  raw = pd.read_csv("/content/텍마/nyt_raw_all.csv", engine='python', on_bad_lines='warn')


6889

In [ ]:
print(f"행: {nyt_raw.shape[0]}, 열: {nyt_raw.shape[1]}")

행: 34462, 열: 10


In [ ]:
!pip install -U spacy tqdm pandas
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.8
    Uninstalling spacy-3.8.8:
      Successfully uninstalled spacy-3.8.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


^C


In [ ]:
import spacy
import pandas as pd
import json
import re
from tqdm import tqdm

# spaCy 모델 로드
# parser + tagger는 남기고, ner는 끔 (속도UP)
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [ ]:
import pandas as pd

file_path = "new_nyt_raw_1993_1998.csv"

df = pd.read_csv(file_path, engine='python', on_bad_lines='warn')

print(f"'{file_path}' 파일이 성공적으로 로드되었습니다. 데이터프레임의 상위 5행은 다음과 같습니다:")
display(df.head())

'new_nyt_raw_1993_1998.csv' 파일이 성공적으로 로드되었습니다. 데이터프레임의 상위 5행은 다음과 같습니다:


,doc_id,doc_type,date,title,source,language,raw_text,web_url,section_name,news_desk
0,NEWS_1993-02-06_1177,news,1993-02-06,JOBLESS RATE HITS A 25-YEAR RECORD IN NEW YORK...,The New York Times,en,JOBLESS RATE HITS A 25-YEAR RECORD IN NEW YORK...,https://www.nytimes.com/1993/02/06/business/jo...,Business Day,Financial Desk
1,NEWS_1993-02-06_1198,news,1993-02-06,BUSINESS DIGEST,The New York Times,en,BUSINESS DIGEST The Economy The Economy Ne...,https://www.nytimes.com/1993/02/06/business/bu...,Business Day,Financial Desk
2,NEWS_1993-05-15_3521,news,1993-05-15,U.S. Industry's Sluggish Pace,The New York Times,en,U.S. Industry's Sluggish Pace People have been...,https://www.nytimes.com/1993/05/15/business/us...,Business Day,Financial Desk
3,NEWS_1993-07-08_1460,news,1993-07-08,Treasury Yields Stay Near Lows,The New York Times,en,Treasury Yields Stay Near Lows Interest rates ...,https://www.nytimes.com/1993/07/08/business/cr...,Business Day,Financial Desk
4,NEWS_1993-08-20_4857,news,1993-08-20,U.S. Tries To Halt Fall Of Dollar,The New York Times,en,U.S. Tries To Halt Fall Of Dollar The Clinton ...,https://www.nytimes.com/1993/08/20/business/cu...,Business Day,Financial Desk


In [ ]:
import numpy as np

def split_doc_into_sentences(doc):
    """
    spaCy Doc → 문장 리스트 (spaCy Span)
    """
    return [sent for sent in doc.sents if sent.text.strip()]

def process_sentence_spacy(sent):
    """
    spaCy Span → (tokens, pos_tags, cleaned_text)
    cleaned_text는:
      - 소문자
      - 구두점 제거
      - 토큰 중 알파벳/숫자 위주로만 유지
    """
    tokens = []
    pos_tags = []
    cleaned_tokens = []

    for token in sent:
        if token.is_space:
            continue

        tokens.append(token.text)
        pos_tags.append(token.tag_)   # 세부 품사 태그

        # cleaned_text용 토큰
        if token.is_punct:
            continue

        # 알파벳/숫자 위주만 남기고 소문자화
        t = token.lemma_.lower() if token.lemma_ != "-PRON-" else token.text.lower()
        t = re.sub(r"[^a-z0-9\-\.]", "", t)
        if t:
            cleaned_tokens.append(t)

    cleaned_text = " ".join(cleaned_tokens)
    return tokens, pos_tags, cleaned_text

In [ ]:
INPUT_PATH = "/content/nyt_raw_core_candidates.csv" # 이전 단계에서 생성된 파일을 입력으로 사용
OUTPUT_PATH = "/content/nyt_sentences_processed.csv" # 처리된 문장을 저장할 새 파일 이름

chunksize = 50  # 한 번에 처리할 문서 수 (너무 크면 메모리 ↑, 너무 작으면 느려짐)
first = True
total_docs = 0
total_sents = 0

chunk_iter = pd.read_csv(INPUT_PATH, chunksize=chunksize, iterator=True, engine='python', on_bad_lines='warn')

for df_chunk in tqdm(chunk_iter, desc="Processing docs"):
    # raw_text가 NaN인 경우 빈 문자열로 치환
    df_chunk["raw_text"] = df_chunk["raw_text"].fillna("")

    texts = df_chunk["raw_text"].tolist()
    docs = list(nlp.pipe(texts, batch_size=16))  # spaCy 병렬 처리

    rows = []

    for row_idx, (idx, row) in enumerate(df_chunk.iterrows()):
        doc_id   = row["doc_id"]
        doc_type = row.get("doc_type", "news")
        date     = row.get("date", "")
        title    = row.get("title", "")
        source   = row.get("source", "")
        language = row.get("language", "en")
        raw_text = row.get("raw_text", "")

        total_docs += 1

        doc = docs[row_idx]
        sents = split_doc_into_sentences(doc)

        if not sents:
            continue

        for sent_id, sent in enumerate(sents):
            tokens, pos_tags, cleaned = process_sentence_spacy(sent)

            # cleaned_text가 너무 짧으면 (예: 1~2단어) 스킵해도 됨
            if len(cleaned.split()) < 3:
                continue

            total_sents += 1

            rows.append({
                "doc_id": doc_id,
                "doc_type": doc_type,
                "date": date,
                "title": title,
                "source": source,
                "language": language,
                "raw_text": raw_text,

                "sent_id": sent_id,
                "text": sent.text.strip(),
                "tokens": json.dumps(tokens, ensure_ascii=False),
                "pos_tags": json.dumps(pos_tags),
                "cleaned_text": cleaned,
            })

    if rows:
        out_df = pd.DataFrame(rows)
        mode = "w" if first else "a"
        header = first
        out_df.to_csv(OUTPUT_PATH, mode=mode, header=header, index=False)
        first = False

print("총 문서 수:", total_docs)
print("총 문장 수:", total_sents)
print("저장 위치:", OUTPUT_PATH)

Processing docs: 138it [02:23,  1.04s/it]

총 문서 수: 6889
총 문장 수: 32326
저장 위치: /content/nyt_sentences_processed.csv


In [ ]:
!pip install -U spacy pandas tqdm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.8
    Uninstalling spacy-3.8.8:
      Successfully uninstalled spacy-3.8.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to 

In [ ]:
import spacy
import pandas as pd
import json
import re
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm", disable=["ner"])  # ner 끄고 속도 ↑

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving old_nyt_raw_all.csv to old_nyt_raw_all.csv


In [ ]:
df = pd.read_csv(list(uploaded.keys())[0])
print(df.head())
print(len(df))

                 doc_id doc_type        date  \
0  NEWS_1993-01-31_6885     news  1993-01-31   
1  NEWS_1993-01-31_6886     news  1993-01-31   
2  NEWS_1993-01-31_6901     news  1993-01-31   
3  NEWS_1993-01-31_6917     news  1993-01-31   
4  NEWS_1993-01-31_6922     news  1993-01-31   

                                             title              source  \
0                        Publishing At Their Peril  The New York Times   
1    County's Data Systems At Yonkers's Fingertips  The New York Times   
2          Hunt for Highway Killer Is Undiminished  The New York Times   
3  Viewpoints;\nThe Monetary Path to Full Recovery  The New York Times   
4                  Jeremy Weithas, M. J. Minnetian  The New York Times   

  language                                           raw_text  \
0       en  Publishing At Their Peril Prime Minister John ...   
1       en  County's Data Systems At Yonkers's Fingertips ...   
2       en  Hunt for Highway Killer Is Undiminished Along ...   
3     

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34463 entries, 0 to 34462
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   doc_id        34463 non-null  object
 1   doc_type      34463 non-null  object
 2   date          34463 non-null  object
 3   title         34463 non-null  object
 4   source        34463 non-null  object
 5   language      34463 non-null  object
 6   raw_text      34463 non-null  object
 7   web_url       34463 non-null  object
 8   section_name  34459 non-null  object
 9   news_desk     31763 non-null  object
dtypes: object(10)
memory usage: 2.6+ MB


In [ ]:
df = df.drop_duplicates()
df = df.drop_duplicates(subset=["title", "date"], keep="first")

# 제목이 정확히 "Key Rates" 인 행만 제거
df = df[df["title"].str.strip().ne("Key Rates")]

df = df.reset_index(drop=True)
print("중복 제거 후:", len(df))

중복 제거 후: 32504


In [ ]:
#섹션제거후
print("중복 제거 후:", len(df))


중복 제거 후: 11017


In [ ]:
df.head(10)

,doc_id,doc_type,date,title,source,language,raw_text,web_url,section_name,news_desk
0,NEWS_1993-01-31_6917,news,1993-01-31,Viewpoints;\nThe Monetary Path to Full Recovery,The New York Times,en,Viewpoints;\nThe Monetary Path to Full Recover...,https://www.nytimes.com/1993/01/31/business/vi...,Business Day,Financial Desk
1,NEWS_1993-01-31_6926,news,1993-01-31,CLINTON WEIGHING FREEZE OR NEW TAX ON SOCIAL S...,The New York Times,en,CLINTON WEIGHING FREEZE OR NEW TAX ON SOCIAL S...,https://www.nytimes.com/1993/01/31/us/clinton-...,U.S.,National Desk
2,NEWS_1993-01-31_6950,news,1993-01-31,This Week's Numbers,The New York Times,en,This Week's Numbers Last Con...,https://www.nytimes.com/1993/01/31/business/th...,Business Day,Financial Desk
3,NEWS_1993-01-31_7031,news,1993-01-31,Stanching the Loss of Good Jobs,The New York Times,en,Stanching the Loss of Good Jobs THE layoff ann...,https://www.nytimes.com/1993/01/31/business/st...,Business Day,Financial Desk
4,NEWS_1993-01-31_7109,news,1993-01-31,Viewpoints;\nIf Domestic Work Paid Well . . .,The New York Times,en,Viewpoints;\nIf Domestic Work Paid Well . . . ...,https://www.nytimes.com/1993/01/31/business/vi...,Business Day,Financial Desk
5,NEWS_1993-01-31_7126,news,1993-01-31,Mutual Funds; Those Wacky Little Stocks Abroad,The New York Times,en,Mutual Funds; Those Wacky Little Stocks Abro...,https://www.nytimes.com/1993/01/31/business/mu...,Business Day,Financial Desk
6,NEWS_1993-01-31_7163,news,1993-01-31,SOME SEE RICHES IN TROUBLED LOANS,The New York Times,en,SOME SEE RICHES IN TROUBLED LOANS For the firs...,https://www.nytimes.com/1993/01/31/us/some-see...,U.S.,National Desk
7,NEWS_1993-01-31_7198,news,1993-01-31,Croats Are Easing the Danger of Damaged Dam's ...,The New York Times,en,Croats Are Easing the Danger of Damaged Dam's ...,https://www.nytimes.com/1993/01/31/world/croat...,World,Foreign Desk
8,NEWS_1993-01-31_7296,news,1993-01-31,"Viewpoints;\nA Tax That Does Double, Triple Duty",The New York Times,en,"Viewpoints;\nA Tax That Does Double, Triple Du...",https://www.nytimes.com/1993/01/31/business/vi...,Business Day,Financial Desk
9,NEWS_1993-02-01_0019,news,1993-02-01,Investors Now Expect Inflation to Remain Low,The New York Times,en,Investors Now Expect Inflation to Remain Low I...,https://www.nytimes.com/1993/02/01/business/cr...,Business Day,Financial Desk


In [ ]:
import spacy
import pandas as pd

# spaCy 로드
nlp = spacy.load("en_core_web_sm")

def split_sentences(text):
    if not isinstance(text, str) or not text.strip():
        return []
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

# 문장 리스트 생성 (raw_text 컬럼 사용)
df["sentences"] = df["raw_text"].apply(split_sentences)

# 문장 단위로 펼치기
df_sent = df.explode("sentences").reset_index(drop=True)

# 문장 번호 부여 (date 컬럼 사용)
df_sent["sent_id"] = df_sent.groupby(["title", "date"]).cumcount()

# 컬럼 정리
df_sent = df_sent.rename(columns={"sentences": "sentence"})

In [ ]:
df.head()

,doc_id,doc_type,date,title,source,language,raw_text,web_url,section_name,news_desk,sentences
0,NEWS_1993-01-31_6917,news,1993-01-31,Viewpoints;\nThe Monetary Path to Full Recovery,The New York Times,en,Viewpoints;\nThe Monetary Path to Full Recover...,https://www.nytimes.com/1993/01/31/business/vi...,Business Day,Financial Desk,[Viewpoints;\nThe Monetary Path to Full Recove...
1,NEWS_1993-01-31_6926,news,1993-01-31,CLINTON WEIGHING FREEZE OR NEW TAX ON SOCIAL S...,The New York Times,en,CLINTON WEIGHING FREEZE OR NEW TAX ON SOCIAL S...,https://www.nytimes.com/1993/01/31/us/clinton-...,U.S.,National Desk,[CLINTON WEIGHING FREEZE OR NEW TAX ON SOCIAL ...
2,NEWS_1993-01-31_6950,news,1993-01-31,This Week's Numbers,The New York Times,en,This Week's Numbers Last Con...,https://www.nytimes.com/1993/01/31/business/th...,Business Day,Financial Desk,[This Week's Numbers Last Co...
3,NEWS_1993-01-31_7031,news,1993-01-31,Stanching the Loss of Good Jobs,The New York Times,en,Stanching the Loss of Good Jobs THE layoff ann...,https://www.nytimes.com/1993/01/31/business/st...,Business Day,Financial Desk,[Stanching the Loss of Good Jobs THE layoff an...
4,NEWS_1993-01-31_7109,news,1993-01-31,Viewpoints;\nIf Domestic Work Paid Well . . .,The New York Times,en,Viewpoints;\nIf Domestic Work Paid Well . . . ...,https://www.nytimes.com/1993/01/31/business/vi...,Business Day,Financial Desk,"[Viewpoints;, If Domestic Work Paid Well . ., ..."


In [ ]:
df["sentence_count"] = df["sentences"].apply(len)
df["sentence_count"].describe()

,sentence_count
count,11017.000000
mean,5.098303
std,43.777384
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,4538.000000


In [ ]:
df_sent.groupby(["title", "date"])["sent_id"].min().describe()

,sent_id
count,11017.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [ ]:
worst = df.nlargest(1, "sentence_count").iloc[0]
print(worst["title"])
print(worst["sentence_count"])
print(worst["raw_text"][:1000])

More Extended Excerpts from Cardinal Bernard F. Law's Deposition
4538
More Extended Excerpts from Cardinal Bernard F. Law's Deposition  1 COMMONWEALTH OF MASSACHUSETTS 2 COUNTY OF MIDDLESEX 3 GREGORY FORD, et al., Plaintiff, 4 Superior Court vs. Civil Action 5 No. 02-0626 BERNARD CARDINAL LAW, a/k/a, 6 CARDINAL BERNARD F. LAW, Defendants. 7 --------------------------------- PAUL W. BUSA, 8 Plaintiff, 9 vs. Civil Action No. 02-0822 10 BERNARD CARDINAL LAW, a/k/a, CARDINAL BERNARD F. LAW, et al. 11 Defendants. ------------------------------------- 12 ANTHONY DRISCOLL, Plaintiff, 13 vs. Civil Action 14 No. 02-1737 BERNARD CARDINAL LAW, a/k/a, 15 CARDINAL BERNARD F. LAW, et al. Defendants. 16 17 THE SECOND DAY OF THE VIDEOTAPED DEPOSITION OF CARDINAL BERNARD F. LAW, a witness called by 18 the Plaintiffs, taken pursuant to the applicable provisions of the Massachusetts Rules of Civil 19 Procedure, before Kathleen M. Silva, Registered Professional Reporter and Notary Public in and 20 for the

In [ ]:
df.loc[df["sentence_count"].idxmax()][["title", "raw_text"]]

,4309
title,More Extended Excerpts from Cardinal Bernard F...
raw_text,More Extended Excerpts from Cardinal Bernard F...


In [ ]:
df = df[df["sentence_count"] < 40].reset_index(drop=True)
print(len(df))

10931


In [ ]:
df["sentence_count"] = df["sentences"].apply(len)
df["sentence_count"].describe()

,sentence_count
count,10931.000000
mean,4.235111
std,3.131748
min,1.000000
25%,3.000000
50%,3.000000
75%,5.000000
max,39.000000


In [ ]:
df = df[~df["title"].str.contains("digest", case=False, na=False)].reset_index(drop=True)
print(len(df))

10737


In [ ]:
import textwrap

# 1. 문장 수가 가장 큰 행 인덱스 찾기
idx = df["sentence_count"].idxmax()

# 2. 해당 행 가져오기
row = df.loc[idx]

# 3. 메타 정보 출력
print("=== INDEX ===")
print(idx)
print("\n=== TITLE ===")
print(row["title"])
print("\n=== DATE ===")
print(row["date"])
print("\n=== SECTION ===")
print(row["section_name"])
print("\n=== SENTENCE COUNT ===")
print(row["sentence_count"])

# 4. 전문 출력 (줄바꿈 정리)
print("\n=== RAW TEXT (FULL) ===\n")
print(textwrap.fill(row["raw_text"], width=120))  # 120자 단위 줄바꿈

=== INDEX ===
997

=== TITLE ===
BUSINESS DIGEST

=== DATE ===
1995-03-25

=== SECTION ===
Business Day

=== SENTENCE COUNT ===
39

=== RAW TEXT (FULL) ===

BUSINESS DIGEST Salomon Brothers Bids For Block of Derivatives Salomon Brothers offered to buy up to $8.1 billion of
derivative securities, clearly deciding that the worst is over for derivatives and they are now probably underpriced.
Salomon Brothers Bids For Block of Derivatives Salomon Brothers offered to buy up to $8.1 billion of derivative
securities, clearly deciding that the worst is over for derivatives and they are now probably underpriced.   Other
securities firms quickly followed Salomon. Within hours, Morgan Stanley said it would bid for the same block of
derivatives, and J. P. Morgan and Merrill Lynch also said they were active in the market. The firms hope to profit by
trading, retaining or restructuring the securities. Page 1.    By using relatively simple techniques to diffuse the risk
of the derivatives immediately

In [ ]:
import spacy
import re
import json

nlp = spacy.load("en_core_web_sm", disable=["ner"])  # NER은 필요없으니 끄고 속도↑

In [ ]:
def process_sentence_spacy(sent):
    """
    한 문장(string) -> (tokens_surface, pos_tags, cleaned_text)
    cleaned_text: lemma 기반, 소문자, 구두점 제거, 특수문자 제거
    """
    doc = nlp(sent)

    tokens_surface = []
    pos_tags = []
    cleaned_tokens = []

    for token in doc:
        if token.is_space:
            continue

        # ① 표면형 토큰 / POS 태깅
        tokens_surface.append(token.text)
        pos_tags.append(token.tag_)   # 세부 POS. 간단히면 token.pos_ 도 가능

        # ② cleaned_text용: lemma → 소문자 → 특수문자 제거
        if token.is_punct:   # 구두점은 cleaned_text에서 제외
            continue

        lemma = token.lemma_ if token.lemma_ != "-PRON-" else token.text
        lemma = lemma.lower()

        # 알파벳/숫자/하이픈/점만 남기기
        lemma = re.sub(r"[^a-z0-9\-\.]", "", lemma)

        if lemma:  # 빈 문자열은 버리기
            cleaned_tokens.append(lemma)

    cleaned_text = " ".join(cleaned_tokens)
    return tokens_surface, pos_tags, cleaned_text

In [ ]:
# 뉴스 → 채권리포트 POS-CLEAN 형식으로 컬럼 맞추기

df_final = df_sent.copy()

# 1) 문서 타입 고정
df_final["doc_type"] = "news"

# 2) sentence → text 로 이름 변경
df_final = df_final.rename(columns={"sentence": "text"})

# 3) 각 문장에 대해 spaCy 처리하여 tokens, pos_tags, cleaned_text 추가
# process_sentence_spacy 함수는 (tokens_surface, pos_tags, cleaned_text) 튜플을 반환합니다.
processed_data = df_final["text"].apply(process_sentence_spacy)

df_final["tokens"] = processed_data.apply(lambda x: json.dumps(x[0], ensure_ascii=False))
df_final["pos_tags"] = processed_data.apply(lambda x: json.dumps(x[1]))
df_final["cleaned_text"] = processed_data.apply(lambda x: x[2])

# 4) 불필요 컬럼 제거
drop_cols = ["section_name", "news_desk"]
df_final = df_final.drop(columns=[c for c in drop_cols if c in df_final.columns])

# 5) 컬럼 순서 통일
col_order = [
    "doc_id",
    "doc_type",
    "date",
    "title",
    "source",
    "language",
    "raw_text",
    "sent_id",
    "text",
    "tokens",
    "pos_tags",
    "cleaned_text"
]

df_final = df_final[col_order]

In [ ]:
print(df_final.head())
print(df_final.columns)

                 doc_id doc_type        date  \
0  NEWS_1993-01-31_6917     news  1993-01-31   
1  NEWS_1993-01-31_6917     news  1993-01-31   
2  NEWS_1993-01-31_6917     news  1993-01-31   
3  NEWS_1993-01-31_6917     news  1993-01-31   
4  NEWS_1993-01-31_6917     news  1993-01-31   

                                             title              source  \
0  Viewpoints;\nThe Monetary Path to Full Recovery  The New York Times   
1  Viewpoints;\nThe Monetary Path to Full Recovery  The New York Times   
2  Viewpoints;\nThe Monetary Path to Full Recovery  The New York Times   
3  Viewpoints;\nThe Monetary Path to Full Recovery  The New York Times   
4  Viewpoints;\nThe Monetary Path to Full Recovery  The New York Times   

  language                                           raw_text  sent_id  \
0       en  Viewpoints;\nThe Monetary Path to Full Recover...        0   
1       en  Viewpoints;\nThe Monetary Path to Full Recover...        1   
2       en  Viewpoints;\nThe Monetary Path t

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10737 entries, 0 to 10736
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   doc_id          10737 non-null  object
 1   doc_type        10737 non-null  object
 2   date            10737 non-null  object
 3   title           10737 non-null  object
 4   source          10737 non-null  object
 5   language        10737 non-null  object
 6   raw_text        10737 non-null  object
 7   web_url         10737 non-null  object
 8   section_name    10737 non-null  object
 9   news_desk       10210 non-null  object
 10  sentences       10737 non-null  object
 11  sentence_count  10737 non-null  int64 
dtypes: int64(1), object(11)
memory usage: 1006.7+ KB


In [ ]:
df_final.to_csv("news_posclean_dirty.csv", index=False)

In [ ]:
from google.colab import files

file_to_download = "news_posclean_dirty.csv"
files.download(file_to_download)

print(f"'{file_to_download}' 파일 다운로드 요청이 완료되었습니다. 브라우저에서 다운로드를 확인해주세요.")

In [ ]:
import re
import pandas as pd

# ---------- 정책어 (Policy Terms) ----------
POLICY_TERMS = [
    "interest rate", "interest rates", "rates",
    "policy rate", "base rate", "benchmark rate", "key rate",
    "monetary policy",
    "mpc", "monetary policy committee",
    "monetary policy board", "mpb",
    "fomc", "federal reserve", "the fed",
    "rate hike", "hike", "raise rates", "tightening", "liftoff",
    "rate cut", "cut rates", "easing",
    "hold rates", "keep rates on hold", "unchanged", "pause",
    "dot plot", "fed dots",
    "summary of economic projections", "sep",
    "forward guidance", "policy guidance", "rate guidance",
]

# ---------- 상관 상위 변수어 (Macro Terms, bucket별) ----------
search_terms = {
    ">=0.7": {
        "Treasuries": [
            "US Treasury yield",
            "10-year Treasury yield",
            "2-year Treasury yield",
            "3-month T-bill yield",
            "Treasury rates",
            "UST yield",
        ],
        "MortgageRate": [
            "30-year fixed mortgage rate",
            "US mortgage rate",
            "mortgage rates",
            "Freddie Mac mortgage rate",
            "PMMS",
        ],
        "Unemployment": [
            "unemployment rate",
            "jobless rate",
            "US unemployment",
            "labor market",
            "U-3 unemployment",
        ],
        "YieldCurve": [
            "yield curve",
            "term spread",
            "yield curve spread",
            "10y-2y spread",
            "10y-3m spread",
            "yield curve inversion",
        ],
    },
    "0.4-0.55": {
        "CorporateBonds": [
            "corporate bond yield",
            "AAA yield",
            "BAA yield",
            "investment-grade yield",
            "credit spread",
            "corporate spread",
        ],
        "IndustrialProduction": [
            "industrial production",
            "industrial production index",
            "IPI",
            "factory output",
            "manufacturing output",
        ],
        "HousingStarts": [
            "housing starts",
            "US housing starts",
            "new residential construction",
            "new housing construction",
        ],
        "BuildingPermits": [
            "building permits",
            "residential building permits",
            "construction permits",
        ],
        "InflationExpectations": [
            "inflation expectations",
            "breakeven inflation",
            "5-year breakeven",
            "10-year breakeven",
            "5y5y inflation expectations",
            "TIPS breakeven",
        ],
    },
}

In [ ]:
def phrase_to_tokens(phrase: str):
    """
    phrase 문자열을 cleaned_text와 같은 스타일의 토큰 리스트로 변환
    예) "interest rate" -> ["interest", "rate"]
    """
    # 알파벳/숫자만 남기고 소문자, 공백 기준 split
    phrase = phrase.lower()
    phrase = re.sub(r"[^a-z0-9\s\-\.]", " ", phrase)
    tokens = [t for t in phrase.split() if t]
    return tokens

# 정책어 phrase들
POLICY_PHRASES = [phrase_to_tokens(p) for p in POLICY_TERMS]

# 상위 변수어 phrase들: 코어/서브로 나눠서
MACRO_PHRASES_CORE = []
MACRO_PHRASES_SUB = []

for bucket, groups in search_terms.items():
    for _, phrases in groups.items():
        for ph in phrases:
            if bucket == ">=0.7":
                MACRO_PHRASES_CORE.append(phrase_to_tokens(ph))
            else:
                MACRO_PHRASES_SUB.append(phrase_to_tokens(ph))

In [ ]:
def find_phrase_positions(tokens, phrase_tokens):
    """
    tokens: 문장 토큰 리스트 ["fed","raise","interest","rate",...]
    phrase_tokens: ["interest","rate"]
    → phrase가 시작하는 index 리스트 반환
    """
    L = len(phrase_tokens)
    if L == 0 or len(tokens) < L:
        return []

    positions = []
    for i in range(len(tokens) - L + 1):
        if tokens[i:i+L] == phrase_tokens:
            positions.append(i)
    return positions

In [ ]:
def has_policy_and_macro_nearby(tokens, K_core=30, K_sub=50):
    """
    tokens: cleaned_text를 split한 토큰 리스트
    반환:
      (True, "core", min_dist)  : 코어 버킷에서 NEAR/30 만족
      (True, "sub",  min_dist)  : 서브 버킷에서 NEAR/50 만족
      (False, None, None)       : 어떤 버킷도 만족하지 않음
    """

    # 1) 정책어 위치들
    policy_positions = []
    for ph in POLICY_PHRASES:
        policy_positions += find_phrase_positions(tokens, ph)

    if not policy_positions:
        return False, None, None

    min_dist_core = None
    min_dist_sub = None

    # 2) 코어 버킷: K_core 이내인지 확인
    for macro_ph in MACRO_PHRASES_CORE:
        macro_positions = find_phrase_positions(tokens, macro_ph)
        for a in policy_positions:
            for b in macro_positions:
                dist = abs(a - b)
                if min_dist_core is None or dist < min_dist_core:
                    min_dist_core = dist

    if min_dist_core is not None and min_dist_core <= K_core:
        return True, "core", min_dist_core

    # 3) 서브 버킷: K_sub 이내인지 확인
    for macro_ph in MACRO_PHRASES_SUB:
        macro_positions = find_phrase_positions(tokens, macro_ph)
        for a in policy_positions:
            for b in macro_positions:
                dist = abs(a - b)
                if min_dist_sub is None or dist < min_dist_sub:
                    min_dist_sub = dist

    if min_dist_sub is not None and min_dist_sub <= K_sub:
        return True, "sub", min_dist_sub

    return False, None, None

In [ ]:
# cleaned_text → 토큰 리스트로 변환하는 간단 함수
def cleaned_to_tokens(cleaned_text: str):
    if not isinstance(cleaned_text, str):
        return []
    return [t for t in cleaned_text.split() if t]

In [ ]:
def neark_classifier(cleaned_text):
    tokens = cleaned_to_tokens(cleaned_text)
    ok, bucket, dist = has_policy_and_macro_nearby(tokens)
    if not ok:
        return pd.Series({"neark_bucket": None, "neark_distance": None})
    else:
        return pd.Series({"neark_bucket": bucket, "neark_distance": dist})

# df_final 기준
neark_df = df_final["cleaned_text"].apply(neark_classifier)
df_final = pd.concat([df_final, neark_df], axis=1)

In [ ]:
df_neark.to_csv("news_posclean_neark.csv", index=False)

In [ ]:
# 'text' 컬럼을 기준으로 중복된 행 제거
df_neark_unique_text = df_neark.drop_duplicates(subset=['text'], keep='first')

# 인덱스 재설정
df_neark_unique_text = df_neark_unique_text.reset_index(drop=True)

print(f"원본 df_neark 데이터프레임 행 수: {len(df_neark)}")
print(f"'text' 컬럼 중복 제거 후 df_neark_unique_text 데이터프레임 행 수: {len(df_neark_unique_text)}")
display(df_neark_unique_text.head())

원본 df_neark 데이터프레임 행 수: 112
'text' 컬럼 중복 제거 후 df_neark_unique_text 데이터프레임 행 수: 94


,doc_id,doc_type,date,title,source,language,raw_text,sent_id,text,tokens,pos_tags,cleaned_text,neark_bucket,neark_distance
0,NEWS_1993-05-15_3521,news,1993-05-15,U.S. Industry's Sluggish Pace,The New York Times,en,U.S. Industry's Sluggish Pace People have been...,2,"Yesterday, in a routine monthly report on indu...","[""Yesterday"", "","", ""in"", ""a"", ""routine"", ""mont...","[""NN"", "","", ""IN"", ""DT"", ""JJ"", ""JJ"", ""NN"", ""IN""...",yesterday in a routine monthly report on indus...,sub,3.0
1,NEWS_1994-05-19_4510,news,1994-05-19,Most Treasury Rates Lower in Active Trading,The New York Times,en,Most Treasury Rates Lower in Active Trading In...,0,Most Treasury Rates Lower in Active Trading In...,"[""Most"", ""Treasury"", ""Rates"", ""Lower"", ""in"", ""...","[""JJS"", ""NNP"", ""NNPS"", ""NNP"", ""IN"", ""JJ"", ""NN""...",most treasury rates lower in active trading in...,core,1.0
2,NEWS_1994-07-09_1592,news,1994-07-09,Dow Gains 20.72 as Fed Doesn't Move on Rates,The New York Times,en,Dow Gains 20.72 as Fed Doesn't Move on Rates S...,0,Dow Gains 20.72 as Fed Doesn't Move on Rates S...,"[""Dow"", ""Gains"", ""20.72"", ""as"", ""Fed"", ""Does"",...","[""NNP"", ""NNPS"", ""CD"", ""IN"", ""NNP"", ""VBZ"", ""RB""...",dow gains 20.72 as fed do not move on rates st...,core,12.0
3,NEWS_1994-08-16_3569,news,1994-08-16,Industrial Production Rises 0.2%,The New York Times,en,Industrial Production Rises 0.2% Despite lower...,0,Industrial Production Rises 0.2% Despite lower...,"[""Industrial"", ""Production"", ""Rises"", ""0.2"", ""...","[""NNP"", ""NNP"", ""VBZ"", ""CD"", ""NN"", ""IN"", ""JJR"",...",industrial production rise 0.2 despite low out...,sub,23.0
4,NEWS_1995-07-08_1378,news,1995-07-08,Bond Prices Fall on Data Showing Jobless Rate ...,The New York Times,en,Bond Prices Fall on Data Showing Jobless Rate ...,0,Bond Prices Fall on Data Showing Jobless Rate ...,"[""Bond"", ""Prices"", ""Fall"", ""on"", ""Data"", ""Show...","[""NN"", ""NNS"", ""VBP"", ""IN"", ""NNP"", ""NNP"", ""NNP""...",bond price fall on data showing jobless rate d...,core,20.0


In [ ]:
# 중복 제거된 데이터를 새 CSV 파일로 저장
output_file_path = "news_posclean_neark_unique_text.csv"
df_neark_unique_text.to_csv(output_file_path, index=False)
print(f"데이터프레임이 '{output_file_path}' 파일로 성공적으로 저장되었습니다.")

데이터프레임이 'news_posclean_neark_unique_text.csv' 파일로 성공적으로 저장되었습니다.


# 11월 18일


In [ ]:
import pandas as pd

df = pd.read_csv("news_posclean_dirty.csv")   # 원본(날아가기 전) 파일 이름

ParserError: Error tokenizing data. C error: Expected 12 fields in line 427, saw 1742


In [ ]:
# 삭제 전 행 수
before = len(df)

# 문서 단위 중복제거 (title + date + text)
df = df.drop_duplicates(subset=["title", "date", "text"])

# 삭제 후 행 수
after = len(df)

# 문서 내부 기준으로 sent_id 다시 매기기
df["sent_id"] = df.groupby(["title", "date"]).cumcount()

# 정렬
df = df.sort_values(by=["date", "title", "sent_id"]).reset_index(drop=True)

print(f"🧹 중복 제거 완료!")
print(f" - 제거 전 문장 수: {before:,}")
print(f" - 제거 후 문장 수: {after:,}")
print(f" - 제거된 문장 수: {before - after:,}")
print(f" - 중복 제거 비율: {(before - after) / before * 100:.2f}%")

🧹 중복 제거 완료!
 - 제거 전 문장 수: 20,880
 - 제거 후 문장 수: 18,787
 - 제거된 문장 수: 2,093
 - 중복 제거 비율: 10.02%


In [ ]:
# 문서 단위 데이터 만들기 (title + date + raw_text 기준으로 고유판별)
df_doc = df[["title", "date", "raw_text"]].drop_duplicates()

print(f"📄 전체 기사 수: {len(df_doc):,}")

📄 전체 기사 수: 4,309


In [ ]:
# 중복 문서 탐지
doc_before = len(df_doc)
df_doc = df_doc.drop_duplicates(subset=["title", "date", "raw_text"])
doc_after = len(df_doc)

print(f"🧹 완전 동일 기사 제거 완료!")
print(f" - 제거 전 기사 수: {doc_before:,}")
print(f" - 제거 후 기사 수: {doc_after:,}")
print(f" - 제거된 기사 수: {doc_before - doc_after:,}")
print(f" - 중복 제거 비율: {(doc_before - doc_after) / doc_before * 100:.2f}%")

🧹 완전 동일 기사 제거 완료!
 - 제거 전 기사 수: 4,309
 - 제거 후 기사 수: 4,309
 - 제거된 기사 수: 0
 - 중복 제거 비율: 0.00%


### 중복 문서 문장 제거 완료

## 잘 가져왔는지 확인

In [ ]:
import re
import pandas as pd

# ---------- 정책어 (Policy Terms) ----------
POLICY_TERMS = [
    "interest rate", "interest rates", "rates",
    "policy rate", "base rate", "benchmark rate", "key rate",
    "monetary policy",
    "mpc", "monetary policy committee",
    "monetary policy board", "mpb",
    "fomc", "federal reserve", "the fed",
    "rate hike", "hike", "raise rates", "tightening", "liftoff",
    "rate cut", "cut rates", "easing",
    "hold rates", "keep rates on hold", "unchanged", "pause",
    "dot plot", "fed dots",
    "summary of economic projections", "sep",
    "forward guidance", "policy guidance", "rate guidance",
]

# ---------- 상관 상위 변수어 (Macro Terms, bucket별) ----------
Macro_terms = {
    ">=0.7": {
        "Treasuries": [
            "US Treasury yield",
            "10-year Treasury yield",
            "2-year Treasury yield",
            "3-month T-bill yield",
            "Treasury rates",
            "UST yield",
        ],
        "MortgageRate": [
            "30-year fixed mortgage rate",
            "US mortgage rate",
            "mortgage rates",
            "Freddie Mac mortgage rate",
            "PMMS",
        ],
        "Unemployment": [
            "unemployment rate",
            "jobless rate",
            "US unemployment",
            "labor market",
            "U-3 unemployment",
        ],
        "YieldCurve": [
            "yield curve",
            "term spread",
            "yield curve spread",
            "10y-2y spread",
            "10y-3m spread",
            "yield curve inversion",
        ],
    },
    "0.4-0.55": {
        "CorporateBonds": [
            "corporate bond yield",
            "AAA yield",
            "BAA yield",
            "investment-grade yield",
            "credit spread",
            "corporate spread",
        ],
        "IndustrialProduction": [
            "industrial production",
            "industrial production index",
            "IPI",
            "factory output",
            "manufacturing output",
        ],
        "HousingStarts": [
            "housing starts",
            "US housing starts",
            "new residential construction",
            "new housing construction",
        ],
        "BuildingPermits": [
            "building permits",
            "residential building permits",
            "construction permits",
        ],
        "InflationExpectations": [
            "inflation expectations",
            "breakeven inflation",
            "5-year breakeven",
            "10-year breakeven",
            "5y5y inflation expectations",
            "TIPS breakeven",
        ],
    },
}

In [ ]:
df['has_policy'] = df['raw_text'].str.contains(
    "|".join(POLICY_TERMS), case=False, regex=True
)

print(df['has_policy'].value_counts(normalize=True))

has_policy
True     0.957098
False    0.042902
Name: proportion, dtype: float64


	•	전체 뉴스 문장의 95.7%가 정책어를 포함하고 있다.
	•	금리, 통화정책, FOMC, Fed, rate hike/cut 같은 핵심 정책 단어가 적어도 1개 이상 포함되어 있음
	•	오직 4.2%만이 정책어 없이 포함된 문장

In [ ]:
df['has_macro'] = df['raw_text'].str.contains(
    "|".join([x for group in search_terms.values() for cat in group.values() for x in cat]),
    case=False, regex=True
)

print(df['has_macro'].value_counts(normalize=True))

has_macro
False    0.891308
True     0.108692
Name: proportion, dtype: float64


	•	전체 문장의 약 10.9%가 상관경제지표(국채·실업률·모기지 등)를 포함
	•	나머지 89.1%는 정책어만 포함 (또는 정치적·제도적 맥락)

In [ ]:
output_file_name = "news_posclean_ver1.csv"
df.to_csv(output_file_name, index=False)
print(f"데이터프레임이 '{output_file_name}' 파일로 성공적으로 저장되었습니다.")

데이터프레임이 'news_posclean_ver1.csv' 파일로 성공적으로 저장되었습니다.
